<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/dphi/datasprint%2324/Data_Sprint_24_hyper_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
X = pd.read_csv('https://raw.githubusercontent.com/Roudranil/competitions/main/dphi/datasprint%2324/X.csv')
y = pd.read_csv('https://raw.githubusercontent.com/Roudranil/competitions/main/dphi/datasprint%2324/y.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Roudranil/competitions/main/dphi/datasprint%2324/test.csv')
submission = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/Tinder_Millennial_Match/sample_submission.csv')

# submission
---

In [ ]:
submission['prediction'] = xgbc.predict(test)
submission.iloc[444, 0] = 0
submission.to_csv('submission9.csv', index=False)